### Library imports

In [25]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, command
from azure.ai.ml.dsl import pipeline 
from azure.ai.ml.entities import JobSchedule, RecurrencePattern, RecurrenceTrigger
from azureml.pipeline.core import TimeZone

import datetime

### 1. Get handle to workspace

In [26]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="27a6aae6-ce60-4ae4-a06e-cfe9c1e824d4",
    resource_group_name="RG-ADA-MLOPS-POC",
    workspace_name="azu-ml-ada-mlops-poc",
)

### 2. Configure the data drift detection script as a pipeline job

In [28]:
from data_drift.data_drift_check import data_drift_component

@pipeline(default_compute = 'shared-compute-poc')

def data_drift_pipeline_definition():
    dd_node = data_drift_component()

pipeline_job = data_drift_pipeline_definition()

ModuleNotFoundError: No module named 'data_drift'

### 3. Create the schedule

In [20]:
schedule_name = "data-dift-schedule"

recurrence_trigger = RecurrenceTrigger(
    frequency="minute",
    interval=20,
    # schedule=RecurrencePattern(hminutes=[30]),
    #start_time=schedule_start_time,
    time_zone=TimeZone.UTC,
)

job_schedule = JobSchedule(
    name=schedule_name, trigger=recurrence_trigger, create_job=pipeline_job
)

### 4. Start the schedule

In [22]:
job_schedule = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()
print(job_schedule)

Uploading alejandro.donaire (5.37 MBs): 100%|██████████| 5365640/5365640 [00:08<00:00, 656446.67it/s] 




ValidationException: Unsupported job type 'CommandJob' in schedule data-dift-schedule.